# Set up software libraries
Authenticate and import as necessary.

In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Mount Google Drive to Google Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Import, authenticate, and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=oNR3N6v4dHF13cwxaQNU9okLy7tO1CAyfXTJMkWoTeE&tc=9oCecXP7THS8ccdiy7JuQziW4hcNdnHioG9JWJqxMnU&cc=yc26lZVl3Z0wLxMEbTfN3ViUMmFOSm_zSmKjcaUMP0U

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk420bw9QG4xUyBNSJG9K26dOQxrLmcYYdhgLFJzD9iZPHHk4eD0Xo4

Successfully saved authorization token.


In [ ]:
# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)

2.12.0


In [ ]:
# Uninstall tensorflow first
!pip uninstall tensorflow -y
# And install again (otherwise it didn't want to connect to a GPU)
!pip3 install tensorflow

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.6 MB/s eta 0:00:00


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11562489560683057480
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14343274496
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 340370195534398092
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
# Check GPU
print('GPU check (should be: /device:GPU:0):')
tf.test.gpu_device_name()

GPU check (should be: /device:GPU:0):


'/device:GPU:0'

In [ ]:
# Import necessary Python packages
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import matplotlib.colors as clr
import pandas as pd

from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.python.keras import optimizers

In [ ]:
# Set seed
tf.random.set_seed(1)

# Set variables

Insert details of the cloud storage bucket and folders, and specify the variables, size and shapes of inputs to the model.

In [ ]:
# Folder with training, validation, and testing data
folder = 'EE_Exports/Shackleton_500mRes_2March2023/'

# Name of cloud storage bucket
bucket = 'ee-surfacemelt'

# Folder and name of trained model
model_folder = 'Models'
model_name = 'AttUnet_500mShackletonTrained'

# Specify model training parameters.
BATCH_SIZE = 32
EPOCHS = 30
SIDE = 64

# Select training, validation, and testing data
Depending on the desired test, a certain melt season or region could be excluded.

In [ ]:
filesList = !gsutil ls 'gs://'{bucket}'/'{folder}
print('All files in folder:', filesList)

trainFilePrefix = 'TrainingPatches'
valFilePrefix = 'ValidationPatches'

meltseason1617 = ['201611', '201612', '201701', '201702', '201703']
meltseason1718 = ['201711', '201712', '201801', '201802', '201803']
meltseason1819 = ['201811', '201812', '201901', '201902', '201903']
meltseason1920 = ['201911', '201912', '202001', '202002', '202003']
meltseason2021 = ['202011', '202012', '202101', '202102', '202103']


R2_R3_R4 = ['R2', 'R3', 'R4',]
R1_R3_R4 = ['R1', 'R3', 'R4',]
R1_R2_R4 = ['R1', 'R2', 'R4',]
R1_R2_R3 = ['R1', 'R2', 'R3',]

All files in folder: ['gs://ee-surfacemelt/EE_Exports/Shackleton_500mRes_2March2023/', 'gs://ee-surfacemelt/EE_Exports/Shackleton_500mRes_2March2023/TrainingPatchesR1_1_1_1_1_S1A_EW_GRDM_1SSH_20161218T132546_20161218T132650_014434_017685_3ECA_S1A_EW_GRDM_1SSH_20161218T132546_20161218T132650_014434_017685_3ECA.tfrecord.gz', 'gs://ee-surfacemelt/EE_Exports/Shackleton_500mRes_2March2023/TrainingPatchesR1_1_1_1_1_S1A_EW_GRDM_1SSH_20170111T132544_20170111T132649_014784_01813D_10E6_S1A_EW_GRDM_1SSH_20170111T132544_20170111T132649_014784_01813D_10E6.tfrecord.gz', 'gs://ee-surfacemelt/EE_Exports/Shackleton_500mRes_2March2023/TrainingPatchesR1_1_1_1_1_S1A_EW_GRDM_1SSH_20170123T132544_20170123T132648_014959_0186B7_95EC_S1A_EW_GRDM_1SSH_20170123T132544_20170123T132648_014959_0186B7_95EC.tfrecord.gz', 'gs://ee-surfacemelt/EE_Exports/Shackleton_500mRes_2March2023/TrainingPatchesR1_1_1_1_1_S1A_EW_GRDM_1SSH_20170204T132544_20170204T132629_015134_018C06_D48C_S1A_EW_GRDM_1SSH_20170204T132544_20170204T1

In [ ]:
# Select test melt season
TestingMeltSeason = meltseason1617

trainFilePath = []
valFilePath = []
testFilePath = []

## Select training files
for line in filesList:
    if trainFilePrefix in line:
        for keyword in TestingMeltSeason:
          if keyword not in line:
            trainFilePath.append(line)
print(np.shape(trainFilePath))

## Select validation files
for line in filesList:
    if valFilePrefix in line:
        for keyword in TestingMeltSeason:
          if keyword not in line:
            valFilePath.append(line)
print(np.shape(valFilePath))

## Select testing files
for line in filesList:
    for keyword in TestingMeltSeason:
        if keyword in line:
          testFilePath.append(line)
print(np.shape(testFilePath))

print(trainFilePath)

(4017,)
(4017,)
(136,)
['gs://ee-surfacemelt/EE_Exports/Shackleton_500mRes_2March2023/TrainingPatchesR1_1_1_1_1_S1A_EW_GRDM_1SSH_20161218T132546_20161218T132650_014434_017685_3ECA_S1A_EW_GRDM_1SSH_20161218T132546_20161218T132650_014434_017685_3ECA.tfrecord.gz', 'gs://ee-surfacemelt/EE_Exports/Shackleton_500mRes_2March2023/TrainingPatchesR1_1_1_1_1_S1A_EW_GRDM_1SSH_20161218T132546_20161218T132650_014434_017685_3ECA_S1A_EW_GRDM_1SSH_20161218T132546_20161218T132650_014434_017685_3ECA.tfrecord.gz', 'gs://ee-surfacemelt/EE_Exports/Shackleton_500mRes_2March2023/TrainingPatchesR1_1_1_1_1_S1A_EW_GRDM_1SSH_20161218T132546_20161218T132650_014434_017685_3ECA_S1A_EW_GRDM_1SSH_20161218T132546_20161218T132650_014434_017685_3ECA.tfrecord.gz', 'gs://ee-surfacemelt/EE_Exports/Shackleton_500mRes_2March2023/TrainingPatchesR1_1_1_1_1_S1A_EW_GRDM_1SSH_20161218T132546_20161218T132650_014434_017685_3ECA_S1A_EW_GRDM_1SSH_20161218T132546_20161218T132650_014434_017685_3ECA.tfrecord.gz', 'gs://ee-surfacemelt/EE_

# Import and read TFRecord files

Load the data exported from Earth Engine into a tf.data.Dataset.

In [ ]:
def input_fn(fileNames, numEpochs=None, shuffle=True, batchSize=BATCH_SIZE, side=SIDE):
  # Read `TFRecordDatasets`
  dataset = tf.data.TFRecordDataset(fileNames, compression_type='GZIP')

  # Names of the features
  feature_columns = {
    'melt_ascat': tf.io.FixedLenFeature([side, side], dtype=tf.float32),
    'melt_SSMIS': tf.io.FixedLenFeature([side, side], dtype=tf.float32),
    'melt_S1_climatology': tf.io.FixedLenFeature([side, side], dtype=tf.float32),
    'elevation': tf.io.FixedLenFeature([side, side], dtype=tf.float32),
    'melt_S1': tf.io.FixedLenFeature([side, side], dtype=tf.float32)}

  # Make a parsing function
  def parse(example_proto):
    parsed_features = tf.io.parse_single_example(example_proto, feature_columns)
    parsed_features = {key:value[0:side,0:side] for key,value in parsed_features.items()}
    labels = parsed_features.pop('melt_S1')
    return parsed_features, tf.cast(labels, tf.int32)

  # Passing of FeatureColumns to a 4D tensor
  def stack_images(features,label):
    nfeat = tf.transpose(tf.squeeze(tf.stack(list(features.values()))))
    nlabel = (tf.transpose(label))[:,:,tf.newaxis]
    return nfeat, nlabel

  dataset = dataset.map(parse, num_parallel_calls=4)
  dataset = dataset.map(stack_images, num_parallel_calls=4)

  if shuffle:
    dataset = dataset.shuffle(buffer_size = batchSize * 10)
  dataset = dataset.batch(batchSize)
  dataset = dataset.repeat(numEpochs)

  return dataset

## Training data



In [ ]:
training = input_fn(trainFilePath, numEpochs=1, shuffle=True, batchSize=BATCH_SIZE, side=SIDE)

print('Training dataset: ', training)

Training dataset:  <_RepeatDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 64, 64, 1), dtype=tf.int32, name=None))>


## Validation data

Now do the same thing to get a validation dataset. Note that unlike the training dataset, the evaluation dataset has a batch size of 1, is not repeated and is not shuffled.

In [ ]:
validation = input_fn(valFilePath, numEpochs=1, shuffle=False, batchSize=1, side=SIDE)

print('Validation dataset: ', validation)

Validation dataset:  <_RepeatDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 64, 64, 1), dtype=tf.int32, name=None))>


## Testing data

Same settings as for the validation dataset are used for the testing dataset.

In [ ]:
test = input_fn(testFilePath, numEpochs=1, shuffle=False, batchSize=1, side=SIDE)

print('Testing dataset: ', test)

Testing dataset:  <_RepeatDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 64, 64, 1), dtype=tf.int32, name=None))>


# Calculate class weights

In [ ]:
training_ytrue_neg = len(training_ytrue)-np.sum(training_ytrue)
training_ytrue_pos = np.sum(training_ytrue)
training_ytrue_total = len(training_ytrue)
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(training_ytrue_total, training_ytrue_pos, 100 * training_ytrue_pos / training_ytrue_total))

# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / training_ytrue_neg) * (training_ytrue_total / 2.0)
weight_for_1 = (1 / training_ytrue_pos) * (training_ytrue_total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

def add_sample_weights(image, label):
  # The weights for each class, with the constraint that:
  #     sum(class_weights) == 1.0
  class_weights = tf.constant([weight_for_0, weight_for_1])
  class_weights = class_weights/tf.reduce_sum(class_weights)

  # Create an image of `sample_weights` by using the label at each pixel as an
  # index into the `class weights` .
  sample_weights = tf.gather(class_weights, indices=tf.cast(label, tf.int32))

  return image, label, sample_weights

# U-NET model

Here we use the Keras implementation of the U-Net model. The U-Net model takes 32x32 pixel patches as input and outputs per-pixel class probability.

The created convolutional neural network model has:
- 5 encoder layers;
- 5 decoder layer
- 1 output layer.

The encoder layer is composed of a linear stack of `Conv`, `BatchNorm`, and `Relu` operations followed by a `MaxPool`. Each `MaxPool` will reduce the spatial resolution of our feature map by a factor of 2. We keep track of the outputs of each block as we feed these high-resolution feature maps with the decoder portion.

The decoder layer is comprised of `UpSampling2D`, `Conv`, `BatchNorm`, and `Relu`. Note that we concatenate the feature map of the same size on the decoder side. Finally, we add a final `Conv` operation that performs a convolution along the channels for each individual pixel (kernel size of (1, 1)) that outputs our final segmentation mask in grayscale.

## Metrics, optimizer, loss function

In [ ]:
# Metrics
METRICS = [
      metrics.TruePositives(name='tp'),
      metrics.FalsePositives(name='fp'),
      metrics.TrueNegatives(name='tn'),
      metrics.FalseNegatives(name='fn'),
      metrics.BinaryAccuracy(name='accuracy'),
      metrics.Precision(name='precision'),
      metrics.Recall(name='recall'),
      metrics.AUC(name='auc'),
      metrics.AUC(name='prc', curve='PR')]

# Optimizer
OPTIMIZER = 'ADAM'

# Loss function
LOSS = 'binary_crossentropy'

## Designing the model

In [ ]:
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.optimizer_v2.adam import Adam

In [ ]:
class attention_unet():
  def __init__(self,img_rows=SIDE,img_cols=SIDE):
    self.img_rows=img_rows
    self.img_cols=img_cols
    self.img_shape=(self.img_rows,self.img_cols,4)
    self.df=32
    self.uf=32

  def build_unet(self):
    def conv2d(layer_input,filters,dropout_rate=0,bn=False):
      d=layers.Conv2D(filters,kernel_size=(3,3),strides=(1,1),padding='same')(layer_input)
      d=layers.Activation('relu')(d)
      d=layers.Conv2D(filters,kernel_size=(3,3),strides=(1,1),padding='same')(d)
      d=layers.Activation('relu')(d)

      if dropout_rate:
        d=layers.Dropout(dropout_rate)(d)

      return d

    def deconv2d(layer_input,filters,bn=False):
      u=layers.UpSampling2D((2,2))(layer_input)
      u=layers.Conv2D(filters,kernel_size=(3,3),strides=(1,1),padding='same')(u)
      u=layers.Activation('relu')(u)

      return u

    def attention_block(F_g,F_l,F_int,bn=False):
      g=layers.Conv2D(F_int,kernel_size=(1,1),strides=(1,1),padding='valid')(F_g)
      x=layers.Conv2D(F_int,kernel_size=(1,1),strides=(1,1),padding='valid')(F_l)
      psi=layers.Add()([g,x])
      psi=layers.Activation('relu')(psi)

      psi=layers.Conv2D(1,kernel_size=(1,1),strides=(1,1),padding='valid')(psi)
      psi=layers.Activation('sigmoid')(psi)

      return layers.Multiply()([F_l,psi])

    inputs=layers.Input(shape=self.img_shape)

    conv1=conv2d(inputs,self.df)
    pool1=layers.MaxPooling2D((2,2))(conv1)

    conv2=conv2d(pool1,self.df*2,bn=True)
    pool2=layers.MaxPooling2D((2,2))(conv2)

    conv3=conv2d(pool2,self.df*4,bn=True)
    pool3=layers.MaxPooling2D((2,2))(conv3)

    conv4=conv2d(pool3,self.df*8,dropout_rate=0.5,bn=True)
    pool4=layers.MaxPooling2D((2,2))(conv4)

    conv5=conv2d(pool4,self.df*16,dropout_rate=0.5,bn=True)

    up6=deconv2d(conv5,self.uf*8,bn=True)
    conv6=attention_block(up6,conv4,self.uf*8,bn=True)
    up6=layers.Concatenate()([up6,conv6])
    conv6=conv2d(up6,self.uf*8)

    up7=deconv2d(conv6,self.uf*4,bn=True)
    conv7=attention_block(up7,conv3,self.uf*4,bn=True)
    up7=layers.Concatenate()([up7,conv7])
    conv7=conv2d(up7,self.uf*4)

    up8=deconv2d(conv7,self.uf*2,bn=True)
    conv8=attention_block(up8,conv2,self.uf*2,bn=True)
    up8=layers.Concatenate()([up8,conv8])
    conv8=conv2d(up8,self.uf*2)

    up9=deconv2d(conv8,self.uf,bn=True)
    conv9=attention_block(up9,conv1,self.uf,bn=True)
    up9=layers.Concatenate()([up9,conv9])
    conv9=conv2d(up9,self.uf)

    outputs=layers.Conv2D(1,kernel_size=(1,1),strides=(1,1),activation='sigmoid')(conv9)

    model=models.Model(inputs=inputs,outputs=outputs)

    model.compile(
    optimizer = optimizers.get(OPTIMIZER),
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

    return model

In [ ]:
m = attention_unet().build_unet()
m.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 64, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 32)   9248        activation[0][0]                 
______________________________________________________________________________________________

## Training the model

You train a Keras model by calling `.fit()` on it.  Here we're going to train for 10 epochs, which is suitable for demonstration purposes.  For production use, you probably want to optimize this parameter, for example through [hyperparameter tuning](https://cloud.google.com/ml-engine/docs/tensorflow/using-hyperparameter-tuning).

Learning rate scheduler: https://towardsdatascience.com/how-to-optimize-learning-rate-with-tensorflow-its-easier-than-you-think-164f980a7c7b

In [ ]:
# Create list of callbacks for (1) learning rate scheduler, and (2) earlier stopping
early_stopping =  tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

callbacks_list = [early_stopping]

In [ ]:
# Fit model
history = m.fit(
     x=training.map(add_sample_weights),
     epochs= EPOCHS,
     batch_size = BATCH_SIZE,
     validation_data=validation,
     callbacks = callbacks_list)

Epoch 1/30
1118/1118 [==============================] - 430s 371ms/step - loss: 0.1793 - tp: 28595506.0000 - fp: 29260220.0000 - tn: 81420840.0000 - fn: 7257910.0000 - accuracy: 0.7508 - precision: 0.4943 - recall: 0.7976 - auc: 0.8522 - prc: 0.6839 - val_loss: 0.4482 - val_tp: 2194408.0000 - val_fp: 1481358.0000 - val_tn: 10924440.0000 - val_fn: 1517554.0000 - val_accuracy: 0.8139 - val_precision: 0.5970 - val_recall: 0.5912 - val_auc: 0.8350 - val_prc: 0.6628
Epoch 2/30
1118/1118 [==============================] - 398s 354ms/step - loss: 0.1702 - tp: 28551988.0000 - fp: 25942340.0000 - tn: 84738704.0000 - fn: 7301414.0000 - accuracy: 0.7731 - precision: 0.5239 - recall: 0.7964 - auc: 0.8666 - prc: 0.6968 - val_loss: 0.4228 - val_tp: 2771213.0000 - val_fp: 2703135.0000 - val_tn: 9702663.0000 - val_fn: 940749.0000 - val_accuracy: 0.7739 - val_precision: 0.5062 - val_recall: 0.7466 - val_auc: 0.8586 - val_prc: 0.6871
Epoch 3/30
1118/1118 [==============================] - 348s 310ms/ste

# Save model

In [ ]:
glob_model = 'gs://' + bucket + '/' + model_folder + '/' + model_name

m.save(glob_model, save_format='tf')

/usr/local/lib/python3.9/dist-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [ ]:
# Save model information to Google Drive

outputDir = '/content/drive/My Drive/UNetResults/'

# convert the history.history dict to a pandas DataFrame:
hist_df = pd.DataFrame(history.history)

# # Save to csv:
hist_csv_file = outputDir + model_name + '.csv'
hist_df.to_csv(hist_csv_file)